# Analyze comparator studies versus fitness estimates and deep mutational scanning for spike

In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/SARS2-mut-fitness/lib/python3.9/site-packages', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmp8hhv_h_m/file/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95f\x1a\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c3results_public_2023-05-11/aa_fitness/aa_fitness.csv\x94\x8c/data/comparator_studies/rodriguez_rivas_dca.csv\x94\x8c)data/comparator_studies/maher_drivers.csv\x94\x8c,data/comparator_studies/thadani_learning.csv\x94\x8c?results_public_2023-05-11/dms/dadonaite_ba1_spike/processed.csv\x94\x8c5results_public_2023-05-11/dms/starr_rbd/processed.csv\x94\x8c8results_public_2023-05-11/dms/iketani_mpro/processed.csv\x94\x8c;results_public_2023-05-11/dms/flynn_mpro_2022/processed.csv\x94\x8c;results_public_2023-05-11/dms/flynn_mpro_2023/processed.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\naa_fitness\x94K\x00N\x86\x94\x8c\x13rodriguez_rivas_dca\x94K\x01N\x86\x94\x8c\rmaher_drivers\x94K\x02N\x86\x94\x8c\x10thadani_learning\x94K\x03N\x86\x94\x8c\x13dadonaite_ba1_spike\x94K\x04N\x86\x94\x8c\tstarr_rbd\x94K\x05N\x86\x94\x8c\x0ciketani_mpro\x94K\x06N\x86\x94\x8c\x0fflynn_mpro_2022\x94K\x07N\x86\x94\x8c\x0fflynn_mpro_2023\x94K\x08N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh*\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h0)}\x94\x8c\x05_name\x94h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bh\x16h\nh\x18h\x0bh\x1ah\x0ch\x1ch\rh\x1eh\x0eh h\x0fh"h\x10h$h\x11h&h\x12ub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8cAresults_public_2023-05-11/comparator_studies/comparator_corr.html\x94a}\x94(h\x14}\x94\x8c\tcorr_html\x94K\x00N\x86\x94sh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bhAh>ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94(K\n}\x94(h\x1e}\x94(\x8c\x05study\x94\x8c;https://www.biorxiv.org/content/10.1101/2022.10.13.512056v1\x94\x8c\x0bdescription\x94\x8c\x1dspike (Dadonaite et al, 2022)\x94\x8c\x04gene\x94\x8c\x01S\x94\x8c\x03url\x94\x8c\x8bhttps://raw.githubusercontent.com/dms-vep/SARS-CoV-2_Omicron_BA.1_spike_DMS_mAbs/main/results/muteffects_functional/muteffects_observed.csv\x94\x8c\x0bfilter_cols\x94}\x94\x8c\ntimes_seen\x94K\x03suh }\x94(\x8c\x05study\x94\x8cOhttps://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1010951\x94\x8c\x0bdescription\x94\x8c\x17RBD (Starr et al, 2022)\x94\x8c\x04gene\x94hW\x8c\x03url\x94\x8c\xadhttps://media.githubusercontent.com/media/jbloomlab/SARS-CoV-2-RBD_DMS_Omicron/f76ba2b2bec18ede9fa9da18c9ccc52389b1ba3a/results/final_variant_scores/final_variant_scores.csv\x94uh"}\x94(\x8c\x05study\x94\x8cChttps://www.sciencedirect.com/science/article/pii/S1931312822004024\x94\x8c\x0bdescription\x94\x8c\x1aMpro (Iketani et al, 2022)\x94\x8c\x04gene\x94\x8c\x0bnsp5 (Mpro)\x94\x8c\x03url\x94\x8c\xachttps://raw.githubusercontent.com/alejandrochavezlab/3CL_protease_DMS/dc802ee3ea9d43005ca97a28af078767fd66777c/outputs/results/normalized_to_wt_and_stop/activity_scores.csv\x94\x8c\x06wt_seq\x94\x8c\x0cdata/Mpro.fa\x94uh$}\x94(\x8c\x05study\x94\x8c(https://elifesciences.org/articles/77433\x94\x8c\x0bdescription\x94\x8c\x18Mpro (Flynn et al, 2022)\x94\x8c\x04gene\x94\x8c\x0bnsp5 (Mpro)\x94\x8c\x03url\x94\x8cKhttps://cdn.elifesciences.org/articles/77433/elife-77433-fig2-data1-v2.xlsx\x94\x8c\x06wt_seq\x94\x8c\x0cdata/Mpro.fa\x94uh&}\x94(\x8c\x05study\x94\x8c)https://doi.org/10.1101/2023.03.02.530652\x94\x8c\x0bdescription\x94\x8c\x18Mpro (Flynn et al, 2023)\x94\x8c\x04gene\x94\x8c\x0bnsp5 (Mpro)\x94\x8c\x03url\x94\x8cahttps://www.biorxiv.org/content/biorxiv/early/2023/03/02/2023.03.02.530652/DC1/embed/media-1.xlsx\x94uu}\x94(h\x18}\x94(\x8c\x04data\x94\x8c_https://raw.githubusercontent.com/GiancarloCroce/DCA_SARS-CoV-2/main/data/data_dca_proteome.csv\x94\x8c\x04name\x94\x8c\x1cRodriguez-Rivas et al (2022)\x94uh\x1a}\x94(\x8c\x04data\x94\x8c>https://www.science.org/doi/suppl/10.1126/scitranslmed.abk3445\x94\x8c\x04name\x94\x8c\x12Maher et al (2022)\x94uh\x1c}\x94(\x8c\x04data\x94\x8c`https://www.biorxiv.org/content/biorxiv/early/2023/04/12/2022.07.21.501023/DC4/embed/media-4.zip\x94\x8c\x04name\x94\x8c\x14Thadani et al (2023)\x94uue}\x94(h\x14}\x94(\x8c\x12min_expected_count\x94K\x00N\x86\x94\x8c\x0cdms_datasets\x94K\x01N\x86\x94\x8c\x12comparator_studies\x94K\x02N\x86\x94uh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bh\x96K\nh\x98hPh\x9ah\x84ub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x11public_2023-05-11\x94a}\x94(h\x14}\x94\x8c\x03mat\x94K\x00N\x86\x94sh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94b\x8c\x03mat\x94h\xa9ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\x14}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bh\xc0K\x01h\xc2K\x01h\xc4h\xbdub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8cMresults_public_2023-05-11/comparator_studies/analyze_comparator_studies.ipynb\x94a}\x94(h\x14}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bh\xd6h\xd3ub\x8c\x06config\x94}\x94(\x8c\tmat_trees\x94}\x94(\x8c\x11public_2023-05-11\x94\x8c\x83http://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/2023/05/11/public-2023-05-11.all.masked.nextclade.pangolin.pb.gz\x94\x8c\x11public_2023-03-27\x94\x8c\x83http://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/2023/03/27/public-2023-03-27.all.masked.nextclade.pangolin.pb.gz\x94\x8c\x11public_2022-12-18\x94\x8c\x83http://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/2022/12/18/public-2022-12-18.all.masked.nextclade.pangolin.pb.gz\x94\x8c\x11gisaid_2023-03-29\x94\x8cQhttps://hgwdev.gi.ucsc.edu/~angie/2847b92/gisaidAndPublic.2023-03-29.masked.pb.gz\x94u\x8c\x0bcurrent_mat\x94\x8c\x11public_2023-05-11\x94\x8c\tref_fasta\x94\x8cGhttp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/bigZips/wuhCor1.fa.gz\x94\x8c\x07ref_gtf\x94\x8cPhttp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/bigZips/genes/ncbiGenes.gtf.gz\x94\x8c\x0eadd_to_ref_gtf\x94}\x94\x8c\x05ORF9b\x94]\x94(M|nM\xa1oes\x8c\x11min_clade_samples\x94M\x10\'\x8c\x0esample_subsets\x94}\x94(\x8c\x03all\x94\x8c\x01.\x94\x8c\x03USA\x94\x8c\x03USA\x94\x8c\x07England\x94\x8c\x07England\x94u\x8c\x12clade_founder_json\x94\x8cyhttps://raw.githubusercontent.com/neherlab/SC2_variant_rates/7e738194a8c6592082f1caa9a6ca70cb68289790/data/clade_gts.json\x94\x8c\x11clades_to_exclude\x94]\x94\x8c\x0321A\x94a\x8c\x10max_nt_mutations\x94K\x04\x8c\x15max_reversions_to_ref\x94K\x01\x8c\x1fmax_reversions_to_clade_founder\x94K\x01\x8c\x1bexclude_ref_to_founder_muts\x94\x88\x8c\x12site_include_range\x94}\x94(\x8c\x05start\x94M\n\x01\x8c\x03end\x94M\xeasu\x8c\x10sites_to_exclude\x94]\x94(K\x99M}\x04M\x96\x08MI\x0cM\xec\rM\xc2\x0eM\xd2\x0fMo\x18MV\x1fMV"MZ4M{6M\x12YME_MF_MeaM\xfd\x15M\xc3\x1aM\xa0\x1cM\xbfmM\xb2re\x8c\rsite_mask_vcf\x94\x8cvhttps://raw.githubusercontent.com/W-L/ProblematicSites_SARS-CoV2/master/subset_vcf/problematic_sites_sarsCov2.mask.vcf\x94\x8c\x12usher_masked_sites\x94\x8c\x1cdata/usher_masked_sites.yaml\x94\x8c\x1dsynonymous_spectra_min_counts\x94M\x88\x13\x8c\x0eorf1ab_to_nsps\x94}\x94(\x8c\x04nsp1\x94]\x94(K\x01K\xb4e\x8c\x04nsp2\x94]\x94(K\xb5M2\x03e\x8c\x04nsp3\x94]\x94(M3\x03M\xcb\ne\x8c\x04nsp4\x94]\x94(M\xcc\nM\xbf\x0ce\x8c\x0bnsp5 (Mpro)\x94]\x94(M\xc0\x0cM\xf1\re\x8c\x04nsp6\x94]\x94(M\xf2\rM\x13\x0fe\x8c\x04nsp7\x94]\x94(M\x14\x0fMf\x0fe\x8c\x04nsp8\x94]\x94(Mg\x0fM,\x10e\x8c\x04nsp9\x94]\x94(M-\x10M\x9d\x10e\x8c\x05nsp10\x94]\x94(M\x9e\x10M(\x11e\x8c\x0cnsp12 (RdRp)\x94]\x94(M)\x11M\xcc\x14e\x8c\x05nsp13\x94]\x94(M\xcd\x14M%\x17e\x8c\x05nsp14\x94]\x94(M&\x17M4\x19e\x8c\x05nsp15\x94]\x94(M5\x19M\x8e\x1ae\x8c\x05nsp16\x94]\x94(M\x8f\x1aM\xb8\x1beu\x8c\rgene_overlaps\x94}\x94(\x8c\x07exclude\x94]\x94(]\x94(\x8c\x05ORF1a\x94\x8c\x06ORF1ab\x94e]\x94(\x8c\x05ORF7a\x94\x8c\x05ORF7b\x94ee\x8c\x06retain\x94]\x94]\x94(\x8c\x01N\x94\x8c\x05ORF9b\x94eau\x8c\x13fitness_pseudocount\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\x12min_expected_count\x94K\n\x8c\x14clade_corr_min_count\x94J \xa1\x07\x00\x8c\x19aa_fitness_init_ref_clade\x94\x8c\x0319A\x94\x8c!aa_fitness_heatmap_minimal_domain\x94]\x94(J\xfa\xff\xff\xffK\x02e\x8c\x0eclade_synonyms\x94}\x94(\x8c\x0319A\x94\x8c\x01B\x94\x8c\x0320A\x94\x8c\x03B.1\x94\x8c\x0320B\x94\x8c\x05B.1.1\x94\x8c\x0320C\x94\x8c\x07B.1.367\x94\x8c\x0320E\x94\x8c\x07B.1.177\x94\x8c\x0320F\x94\x8c\x03D.2\x94\x8c\x0320G\x94\x8c\x05B.1.2\x94\x8c\x0320I\x94\x8c\x05Alpha\x94\x8c\x0320J\x94\x8c\x05Gamma\x94\x8c\x0321C\x94\x8c\x07Epsilon\x94\x8c\x0321F\x94\x8c\x04Iota\x94\x8c\x0321I\x94\x8c\x05Delta\x94\x8c\x0321J\x94\x8c\x05Delta\x94\x8c\x0321K\x94\x8c\x0cOmicron BA.1\x94\x8c\x0321L\x94\x8c\x0cOmicron BA.2\x94\x8c\x0322A\x94\x8c\x0cOmicron BA.4\x94\x8c\x0322B\x94\x8c\x0cOmicron BA.5\x94\x8c\x0322C\x94\x8c\x11Omicron BA.2.12.1\x94\x8c\x0322D\x94\x8c\x0fOmicron BA.2.75\x94\x8c\x0322E\x94\x8c\x0cOmicron BQ.1\x94u\x8c\x0cdms_datasets\x94hP\x8c\rneher_fitness\x94\x8c\x19data/Neher_aa_fitness.csv\x94\x8c\x14clade_fixed_muts_ref\x94\x8c\x0319A\x94\x8c\x14terminal_pseudocount\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\x19terminal_min_actual_count\x94K\x05\x8c\x04dnds\x94\x8c^https://raw.githubusercontent.com/spond/SARS-CoV-2-variation/master/windowed-sites-fel-all.csv\x94\x8c\x12comparator_studies\x94h\x84\x8c\x08docs_url\x94\x8c-https://jbloomlab.github.io/SARS2-mut-fitness\x94\x8c\x15docs_plot_annotations\x94\x8c\x1fdata/docs_plot_annotations.yaml\x94u\x8c\x04rule\x94\x8c\x1aanalyze_comparator_studies\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8cP/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2023/SARS2-mut-fitness');
######## snakemake preamble end #########


In [2]:
import altair as alt

import numpy

import pandas as pd

import scipy.stats.mstats

In [3]:
# get variables from `snakemake`
aa_fitness_csv = snakemake.input.aa_fitness
corr_html = snakemake.output.corr_html
min_expected_count = snakemake.params.min_expected_count
comparator_studies = snakemake.params.comparator_studies

# get DMS datasets with added entry for data
dms_datasets = snakemake.params.dms_datasets
for dataset in dms_datasets:
    dms_datasets[dataset]["data"] = pd.read_csv(getattr(snakemake.input, dataset))

Read data from comparator studies:

In [4]:
for study, study_d in comparator_studies.items():
    
    datafile = getattr(snakemake.input, study)
    print(f"Reading data from {datafile}")
    
    if study == "rodriguez_rivas_dca":
        comparator_studies[study]["df"] = (
            pd.read_csv(datafile)
            .rename(
                columns={
                    "position_protein": "site",
                    "mutability_score(DCA)": "DCA mutability score",
                }
            )
            .query("protein == 'Spike'")
            .assign(gene="S")
            [["gene", "site", "DCA mutability score"]]
            .query("`DCA mutability score`.notnull()")
            .reset_index(drop=True)
        )   
        
    elif study == "maher_drivers":
        comparator_studies[study]["df"] = (
            pd.read_csv("data/comparator_studies/maher_drivers.csv")
            .assign(
                gene=lambda x: x["Unnamed: 0"].str.split("_").str[0],
                mutation=lambda x: x["Unnamed: 0"].str.split("_", 1).str[1],
            )
            .query("gene == 'Spike'")
            .assign(gene="S")
            .query("mutation.str.fullmatch('[A-Z]\d+[A-Z]')")
            .assign(
                wildtype=lambda x: x["mutation"].str[0],
                site=lambda x: x["mutation"].str[1: -1].astype(int),
                aa=lambda x: x["mutation"].str[-1],
            )
            .rename(columns={"EpiScore_AllLineages": "EpiScore"})
            .groupby(["gene", "site", "aa"], as_index=False)
            .aggregate({"EpiScore": "mean"})
            .reset_index(drop=True)
        )
        
    elif study == "thadani_learning":
        comparator_studies[study]["df"] = (
            pd.read_csv("data/comparator_studies/thadani_learning.csv")
            .rename(columns={"i": "site", "mut": "aa", "eve": "EVE score"})
            .assign(gene="S")
            [["gene", "site", "aa", "EVE score"]]
    
        )

    else:
        raise ValueError(f"invalid {study=}")
    

Reading data from data/comparator_studies/rodriguez_rivas_dca.csv
Reading data from data/comparator_studies/maher_drivers.csv
Reading data from data/comparator_studies/thadani_learning.csv


Read amino-acid fitnesses, and for each site calculate the mean fitness of mutations at each site.

In [5]:
aa_fitness = (
    pd.read_csv(aa_fitness_csv)
    .query("aa != '*'")
    .query("expected_count >= @min_expected_count")
    .rename(columns={"aa_site": "site"})
    .query("gene == 'S'")
    [["gene", "site", "aa", "fitness"]]
)

print("Fitnesses for amino acids:")
display(aa_fitness)

print("\nMean fitnesses for sites:")
site_fitness = (
    aa_fitness
    .groupby(["gene", "site"], as_index=False)
    .aggregate(fitness=pd.NamedAgg("fitness", "mean"))
)
display(site_fitness)

Fitnesses for amino acids:


,gene,site,aa,fitness
61583,S,1,I,-6.92550
61584,S,1,K,-3.37120
61585,S,1,L,-4.09100
61586,S,1,M,0.00000
61588,S,1,T,-4.89210
...,...,...,...,...
70889,S,1272,C,-3.38030
70891,S,1272,F,-2.63510
70892,S,1272,H,-0.42621
70893,S,1272,N,-3.37120



Mean fitnesses for sites:


,gene,site,fitness
0,S,1,-4.044917
1,S,2,-1.128540
2,S,3,-1.626888
3,S,4,-1.097812
4,S,6,-0.626228
...,...,...,...
1253,S,1268,-1.606183
1254,S,1269,-2.951920
1255,S,1270,-1.680775
1256,S,1271,-2.104300


Now for each spike DMS dataset and also get the mean DMS measured value at each site:

In [6]:
dms_dfs = []

for dms_dataset, dms_d in dms_datasets.items():
    
    if dms_d["gene"] != "S":
        continue
   
    dms_df = (
        dms_d["data"]
        .query("(wildtype != '*') and (mutant != '*')")
    )
    
    if dms_df["site"].dtype != int:
        dms_df = (
            dms_df
            .query("site.str.isnumeric()")
            .assign(site=lambda x: x["site"].astype(int))
        )
    
    if not len(dms_df.query("wildtype == mutant")):
        # add wildtypes with effects of zero
        dms_df = pd.concat(
            [
                dms_df,
                (
                    dms_df
                    [["site", "wildtype"]]
                    .drop_duplicates()
                    .assign(mutant=lambda x: x["wildtype"], effect=0)
                ),
            ]
        )
    
    if "filter_cols" in dms_d:
        for c, x in dms_d["filter_cols"].items():
            dms_df = dms_df[dms_df[c] >= x]
            
    # calculate site summary DMS and merge with dN/dS and fitness
    dms_dfs.append(
        dms_df.assign(gene=dms_d["gene"], study=dms_d["description"])
    )
    
dms_df = pd.concat(dms_dfs)[["study", "gene", "site", "mutant", "effect"]].rename(
    columns={"mutant": "aa", "study": "DMS study", "effect": "DMS effect"}
)

print("Deep mutational scanning data:")
display(dms_df)

dms_site_df = (
    dms_df.groupby(["DMS study", "gene", "site"], as_index=False)
    .aggregate({"DMS effect": "mean"})
)

print(f"\nDeep mutational scanning data mean site values:")
dms_site_df

Deep mutational scanning data:


,DMS study,gene,site,aa,DMS effect
0,"spike (Dadonaite et al, 2022)",S,1,I,-2.2303
1,"spike (Dadonaite et al, 2022)",S,1,T,-2.4823
4,"spike (Dadonaite et al, 2022)",S,2,L,0.2451
5,"spike (Dadonaite et al, 2022)",S,2,S,0.3820
6,"spike (Dadonaite et al, 2022)",S,2,V,0.3639
...,...,...,...,...,...
3724,"RBD (Starr et al, 2022)",S,473,Y,0.0000
3743,"RBD (Starr et al, 2022)",S,489,Y,0.0000
3762,"RBD (Starr et al, 2022)",S,495,Y,0.0000
3781,"RBD (Starr et al, 2022)",S,505,Y,0.0000



Deep mutational scanning data mean site values:


,DMS study,gene,site,DMS effect
0,"RBD (Starr et al, 2022)",S,331,-0.258277
1,"RBD (Starr et al, 2022)",S,332,-0.011155
2,"RBD (Starr et al, 2022)",S,333,-0.236654
3,"RBD (Starr et al, 2022)",S,334,-0.218237
4,"RBD (Starr et al, 2022)",S,335,-0.059493
...,...,...,...,...
1435,"spike (Dadonaite et al, 2022)",S,1248,-0.299075
1436,"spike (Dadonaite et al, 2022)",S,1249,-0.014183
1437,"spike (Dadonaite et al, 2022)",S,1250,-0.114000
1438,"spike (Dadonaite et al, 2022)",S,1251,0.137300


Now concatenate everything together into one data frame with comparator study effects, fitness effects, and DMS:

In [10]:
merged_dfs = []

for study, study_d in comparator_studies.items():
    comparator_df = (
        study_d["df"]
        .assign(
            comparator_study=study_d["name"],
            comparator_metric=study_d["df"].columns[-1],
        )
        .rename(columns={study_d["df"].columns[-1]: "comparator score"})
    )
    
    if "aa" in comparator_df.columns:
        comparator_df = (
            comparator_df
            .merge(dms_df, validate="one_to_many")
            .merge(aa_fitness, validate="many_to_one")
            .assign(aa_site=lambda x: x["aa"] + x["site"].astype(str))
            .drop(columns=["site", "aa"])
        )
    else:
        comparator_df = (
            comparator_df
            .merge(dms_site_df, validate="one_to_many")
            .merge(site_fitness, validate="many_to_one")
            .rename(columns={"site": "aa_site"})
        )
        
    merged_dfs.append(comparator_df)
    
merged_df = pd.concat(merged_dfs, ignore_index=True)

merged_df

,gene,aa_site,comparator score,comparator_study,comparator_metric,DMS study,DMS effect,fitness
0,S,33,-0.629800,Rodriguez-Rivas et al (2022),DCA mutability score,"spike (Dadonaite et al, 2022)",-0.205417,-0.314275
1,S,34,-1.305700,Rodriguez-Rivas et al (2022),DCA mutability score,"spike (Dadonaite et al, 2022)",-1.699922,-1.892543
2,S,35,-1.375700,Rodriguez-Rivas et al (2022),DCA mutability score,"spike (Dadonaite et al, 2022)",-1.350217,-1.495669
3,S,36,-1.138800,Rodriguez-Rivas et al (2022),DCA mutability score,"spike (Dadonaite et al, 2022)",-0.097117,-1.077818
4,S,37,-1.422600,Rodriguez-Rivas et al (2022),DCA mutability score,"spike (Dadonaite et al, 2022)",-0.562950,-2.123040
...,...,...,...,...,...,...,...,...
11125,S,Y743,-21.962940,Thadani et al (2023),EVE score,"spike (Dadonaite et al, 2022)",-2.650200,-5.747800
11126,S,Y738,-21.489722,Thadani et al (2023),EVE score,"spike (Dadonaite et al, 2022)",-2.472500,-5.747800
11127,S,Y1032,-21.721045,Thadani et al (2023),EVE score,"spike (Dadonaite et al, 2022)",-2.967200,-4.649200
11128,S,C999,-22.100269,Thadani et al (2023),EVE score,"spike (Dadonaite et al, 2022)",-2.061900,-3.672200
